In [ ]:
using Plots
import Plots: px
theme(:default)
ENV["LINES"] = 20

In [ ]:
include("parser.jl")
include("models.jl")
gpu()
download(SIGDataSet)

In [ ]:
data  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-train-high"));
vocab = Vocabulary(data)
edata = encode(data,vocab);
test  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-test"));
tdata = encode(test,vocab);
dictionary = [parseDataLine(line) for line in  eachline("./data/unimorph/spa/spa") if line != ""];

In [ ]:
vocab.chars.toElement;

In [ ]:
vocab.specialIndices

In [ ]:
trainsfs = unique(map(x->join(x.surface),data));
testsfs  =  unique(map(x->join(x.surface),test)); 
dictsfs  = unique([map(x->join(x.surface),dictionary); trainsfs; testsfs]);

In [ ]:
morph = EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
train_ae!(morph, data, vocab; optim=Adam(), epoch=50)

In [ ]:
morphv =  EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
copytoparams(morphv.encoder, morph.encoder)
copytoparams(morphv.Wμ, morph.Wμ)
copytoparams(morphv.Wσa, morph.Wσa)
copytoparams(morphv.Wμa, morph.Wμa)
copytoparams(morphv.Wσ, morph.Wσ)
copytoparams(morphv.Weaμ, morph.Weaμ)
copytoparams(morphv.Weaσ, morph.Weaσ)
morphv.decoder.embedding = morphv.encoder.embedding
train_vae!(morphv, data, vocab; optim=Adam(), epoch=45, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=3)

In [ ]:
samples = sample(morphv, vocab, edata; N=10000);

In [ ]:
unique(samples[findall([s ∈ trainsfs for s in samples])])

In [ ]:
length(samples[findall([s ∈ trainsfs for s in samples])])

In [ ]:
unique(samples[findall([s ∈ testsfs for s in samples])])

In [ ]:
length(samples[findall([s ∈ testsfs for s in samples])])

In [ ]:
unique(length(samples[findall([s ∈ dictsfs for s in samples])]))

In [ ]:
sampleinter(morphv, vocab, data; N=10)

In [ ]:
au, sigma, mu = calc_au(morphv,tdata; delta=0.01)

In [ ]:
mi = calc_mi(morphv,tdata)

In [ ]:
calc_ppl(morphv,tdata)

In [ ]:
atts = attentions(morph,edata, vocab);

In [ ]:
function getsingle(sfs,exs,perms, αs,i) 
    join(vocab.chars[sfs[i]]), map(e->join(vocab.chars[e]),exs[i][perms[i]]), map(x->x[:,perms[i],i],αs)
end

In [ ]:
i = rand(1:length(atts))
s1,e1,α1 =  getsingle(atts[i]...,rand(1:16))
ys = 1:512
h1 = heatmap(e1, ys, α1.αu; size=(1200,800), xtickfont = font(10, "Halvetica"), title= s1 * ", mu", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);
h2 = heatmap(e1, ys, α1.ασ ;size=(1200,800), xtickfont = font(10, "Halvetica"), title=s1 * ", sigma", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);

In [ ]:
println(s1);println(e1)

In [ ]:
h1

# To-Dos
1. Add unrelated surface forms to the examplars too see whether the attention mappings are meaningful
2. Comparing with Normal Vae
3. Different Sampling Function on LM ($top_k$, temperature)
4. Attention in Decoder
5. Attention to the hidden state sequence, not just to the final state.
6. Metric for testing generation quality?

In [89]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [92]:
morph3=nothing; morphv=nothing; morph=nothing; morph2=nothing; KnetLayers.gc();

In [112]:
morph2 = VAE(length(vocab.chars); H=512, E=16, Z=8)
train_ae!(morph2, data, vocab; optim=Adam(), B=16, epoch=40);

(loss = 24.55620150756836,)
(loss = 19.02036479034424,)
(loss = 16.35387802734375,)
(loss = 14.170919728088379,)
(loss = 12.180779341125488,)
(loss = 10.665629011535644,)
(loss = 9.464852150726319,)
(loss = 8.461235062408447,)
(loss = 7.617784868621826,)
(loss = 6.800411344146728,)
(loss = 6.107074007034302,)
(loss = 5.441792821121216,)
(loss = 5.044499470901489,)
(loss = 4.559214506912231,)
(loss = 4.207478071212768,)
(loss = 3.8297413303375243,)
(loss = 3.556478614616394,)
(loss = 3.3634676050186156,)
(loss = 3.093875938606262,)
(loss = 2.9913766212463377,)
(loss = 2.711427449607849,)
(loss = 2.519268382644653,)
(loss = 2.5123119176864623,)
(loss = 2.281764015197754,)
(loss = 2.2385372722625734,)
(loss = 2.1195120547294617,)
(loss = 1.9896105458259583,)
(loss = 1.9000165726661682,)
(loss = 1.827603787136078,)
(loss = 1.766822146320343,)
(loss = 1.6349496092796325,)
(loss = 1.720503440093994,)
(loss = 1.5374233186721802,)
(loss = 1.5526917066574097,)
(loss = 1.4123271984100343,)
(loss

In [ ]:
morph3 = VAE(length(vocab.chars); H=512, E=16, Z=8)
copytoparams(morph3.encoder, morph2.encoder)
copytoparams(morph3.Wμ, morph2.Wμ)
copytoparams(morph3.Wσ, morph2.Wσ)
copytoparams(morph3.dec_embed, morph3.encoder.embedding)
train_vae!(morph3, data, vocab; B=16, optim=Adam(lr=0.002), epoch=40, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=4)

(kl_weight = 0.1f0, fbr = 4, loss = 16.840670321655274)
(kl_weight = 0.2f0, fbr = 4, loss = 15.982103681945802)
(kl_weight = 0.3f0, fbr = 4, loss = 16.14379832611084)
(kl_weight = 0.4f0, fbr = 4, loss = 16.412051783752442)
(kl_weight = 0.5f0, fbr = 4, loss = 16.642443635559083)
(kl_weight = 0.6f0, fbr = 4, loss = 16.818348011779786)
(kl_weight = 0.70000005f0, fbr = 4, loss = 16.80897166290283)
(kl_weight = 0.8000001f0, fbr = 4, loss = 16.771909648132326)
(kl_weight = 0.9000001f0, fbr = 4, loss = 16.667205836486815)
(kl_weight = 1.0f0, fbr = 4, loss = 16.551623442077638)
(kl_weight = 1.0f0, fbr = 4, loss = 16.398298126220702)
(kl_weight = 1.0f0, fbr = 4, loss = 15.90764072265625)
(kl_weight = 1.0f0, fbr = 4, loss = 15.640142103576661)
(kl_weight = 1.0f0, fbr = 4, loss = 15.331370698547364)
(kl_weight = 1.0f0, fbr = 4, loss = 15.064206332397461)
(kl_weight = 1.0f0, fbr = 4, loss = 14.772065257263183)
(kl_weight = 1.0f0, fbr = 4, loss = 14.541408805847167)


In [ ]:
samples = sample(morph3, vocab, edata; N=10000, useprior=true)

In [ ]:
samples[findall([s ∈ trainsfs for s in samples])]

In [ ]:
samples[findall([s ∈ testsfs for s in samples])]

In [ ]:
samples[findall([s ∈ dictsfs for s in samples])]

In [ ]:
inters = sampleinter(morph3, vocab, data; N=10)

In [ ]:
findall([s ∈ dictsfs for s in inters])

In [ ]:
au, sigma, mu = calc_au(morph3, tdata; delta=0.01)

In [ ]:
mi =  calc_mi(morph3,tdata)

In [ ]:
calc_ppl(morph3,tdata; nsample=500, B=16)

In [138]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [ ]:
model = LSTM_LM(length(vocab.chars); H=768, E=16)

In [ ]:
train_rnnlm!(model, data, vocab; epoch=40, optim=Adam(), B=16)

In [ ]:
 calc_ppllm(model, tdata)

In [ ]:
samples = sample(model, vocab; N=10000)

In [ ]:
samples[findall([s ∈ trainsfs for s in samples])]

In [ ]:
samples[findall([s ∈ testsfs for s in samples])]

In [ ]:
samples[findall([s ∈ dictsfs for s in samples])]

In [ ]:
 calc_ppllm(model, edata)